## Práctica 1. Resolución de problemas con búsqueda 
### Sesión 3. Búsqueda Heurística
### Inteligencia Artificial I    2024/2025

En el notebook de la sesión anterior habéis aprendido, a través de ejemplos resueltos, cómo se representan algunos problemas de juguete como el de las jarras, el problema de los misioneros y el problema del ocho puzzle. Además habéis podido experimentar con los algoritmos de búsqueda exhaustiva (ciega) vistos en clase midiendo las propiedades de los algoritmos.
Después de la sesión anterior debes entender el comportamiento de la búsqueda en los distintos problemas y ser capaz de representar y resolver con búsqueda ciega un problema dado (problema de los relojes, por ejemplo). 

En este notebook veremos el comportamiento de los distintos algoritmos de búsqueda heurística sobre algunos problemas dados. 
Observa los resultados y tiempo de la ejecución de los algoritmos de búsqueda heurística y comparalos con los que ya conoces de búsqueda ciega. 
Observa las propiedades de eficiencia, completitud y optimalidad en cada caso. 
**No hay que entregar este notebook**  
Ejecuta este notebook entendiendo y experimentando con los conceptos aprendidos. Una vez terminado realiza el ejercicio propuesto para entregar (MoVIX) en otro notebook y entregalo a través del campus en la entrega de la Sesión 3. 

In [ ]:
from search import *

In [ ]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
# Puedes hacer tu propia implementación del problema de forma genérica para permitir más personajes.
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema. En este caso el estado inicial y objetivo se pasan como parámetros'''
        Problem.__init__(self, initial, goal)
        # En este caso representamos cada accion con un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido. La comprobación de las precondiciones se realiza en una función auxiliar is_valid
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3 
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:  #si la canoa está en el lado 0
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [ ]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

In [ ]:
depth_first_graph_search(misioneros).solution()

[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1m', (1, 0)),
 ('1m1c', (1, 1))]

In [ ]:
breadth_first_tree_search(misioneros).solution()

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

En este problema (de tamaño de juguete) hemos visto que ninguna de las búsquedas ciega supone un problema de eficiencia (sí de terminación si no se comprueban los ciclos). Puedes ampliar el tamaño del problema (con más personajes) para ver cómo escala la resolución del mismo.

#### Añadimos y probamos heurísticas para el problema de los misioneros.

In [ ]:
# Heurísticas para el problema de los misioneros. Observa sus propiedades teóricas y su implementación. 

def h_1(node):
 s = list(node.state)
 return (s[0] + s[1])/2
        
def h_2(node):
 s = list(node.state)
 return 2*(s[0] + s[1])-s[2]

def h_3(node):
        s = node.state
        n = s[0] + s[1]
        if n == 0: return 0
        if n == 6 and s[2] == 1: return float('inf')
        if n == 1 and s[2] == 0: return 1
        if s[2] == 0: return 2*n-3
        if s[2] == 1: return 2*n

En h_3 se tiene en cuenta que si tenemos n personas en la orilla izquierda y la barca está en la misma, necesitaremos hacer $n-1$ vueltas, luego $2n-3$ viajes. Esto es porque en la última vuelta no necesitamos volver a la orilla izquierda, y claramente se cumple solo para $n>1$. Para $n=1$, la función vale 1.
Hay que destacar que solo hemos definido la heurística para los nodos en los que la barca esté en la izquierda, y deberíamos definirla para todos los nodos. En caso en que la barca esté a la derecha y haya n personas en la izquierda, la heurística valdrá $2*(n+1)-3+1=2*n$, asumiendo que haya alguien en la orilla derecha.
En el caso en el que no haya nadie en la orilla izquierda, la heurística valdrá siempre 0 (se implementa como un caso directo) y en el caso en el que la barca esté en la derecha, haya alguien en la orilla izquierda y no haya nadie en la derecha, el problema será irresoluble y la heurística valdrá $+\infty$.

Recuerda que se usa %%timeit (varias iteraciones) para poder medir tiempos de ejecución pequeños, repitiendo varias veces la misma búsqueda, pero no debes usarlo para búsquedas largas.

In [ ]:
%%time
astar_search(misioneros, h_1).solution()

CPU times: total: 0 ns
Wall time: 0 ns


[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [ ]:
%%timeit
astar_search(misioneros, h_1).solution()

68.9 μs ± 1.03 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
%%timeit
astar_search(misioneros, h_2)

65.9 μs ± 1.33 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
%%timeit
astar_search(misioneros, h_3)

67 μs ± 708 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Observa los tiempos y si hemos mejorado (y cuánto)por utilizar búsqueda heurística respecto a la búsqueda ciega. 
Para todos los problemas debes observar al experimentar con los algoritmos si el resultado obtenido es el esperado según los conceptos teóricos vistos en clase. 

### Puzzle deslizante de 8 y 16 piezas

In [ ]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        return tuple(l)
    
    def h(self, state):
        """ Return the heuristic value for a given state. """
        return 1
    
    
    #Añadido para la parte II de la práctica:
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
    # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
    #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0     

###  Puzle de 16 piezas.
Ten en cuenta que el número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. 

In [ ]:
class Quince_Puzzle(Problem): #Todo análogo al caso anterior
    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2,3):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (0,4,8,12):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (12,13,14,15):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (3,7,11,15):
            accs.append("Mover hueco derecha")
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-4]
            l[pos_hueco-4] = 0
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+4]
            l[pos_hueco+4] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1
    
    #Añadido para la parte II de la práctica:
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
    # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
    #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0    

In [ ]:
####  Vamos a probar el 8-puzzle utilizando el siguiente **estado inicial** 

#              +---+---+---+
#              | 2 | 4 | 3 |
#              +---+---+---+
#              | 1 | 5 | 6 |
#              +---+---+---+
#              | 7 | 8 | H |
#              +---+---+---+
# Hay al menos una solución de longitud 8: 
#['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']

puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
if puzle.check_solvability(puzle.initial):
    sol= breadth_first_graph_search(puzle).solution()
else: 
    sol="Este estado inicial no tiene solucion"
print(sol)

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']


In [ ]:
# Con el mismo estado inicial la DFS es inviable aunque BFS ha encontrado la solución óptima en un tiempo razonable. 
puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
if puzle.check_solvability(puzle.initial):
    sol= depth_first_graph_search(puzle).solution()
else: 
    sol="Este estado inicial no tiene solucion"
print(sol)

KeyboardInterrupt: 

En este ejemplo seguro que la búsqueda heurística mejora el rendimiento. Vamos a probarlo en el siguiente apartado.

También podríamos hacemos algunas pruebas con el puzle de 16 (pero no te recomiendo búsquedas ciegas!!
En el caso del siguiente estado inicial que sí tiene solución: (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) 
Puedes probarlo si quieres pero ten en cuenta que tarda mucho tiempo.

In [ ]:
p15 = Quince_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5, 11, 15, 12, 14, 13, 9, 10))
p15.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5, 11, 15, 12, 14, 13, 9, 10)

In [ ]:
p15 = Quince_Puzzle((2, 7, 3, 1, 6, 4, 8, 0, 5, 11, 15, 12, 14, 13, 9, 10))
if p15.check_solvability(p15.initial):
    sol= breadth_first_graph_search(p15).solution()
else: 
    sol="Este estado inicial no tiene solucion"
print(sol)
# Puzle de 16: la búsqueda en anchura tiene que terminar (dado que el estado inicial es resoluble) porque es un algoritmo completo. No obstante, encontrar la solución sin una heurística es una búsqueda muy ineficiente. Puedes observarlo si tienes mucho tiempo.

KeyboardInterrupt: 

En el tamaño 16 la BFS que funcionó bien en tamaño 8 es también inviable.
La anterior ejecución sabemos que termina porque el estado inicial tiene solución y la búsqueda en anchura es completa, pero ten en cuenta que tarda demasiado. No es viable intentar una búsqueda ciega. 

#### Para el problema del puzle de 8 y de 16 vamos a definir y a probar las funciones heurísticas vistas en clase:

In [ ]:
#### 
# linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
# manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
# max_heuristic(node: maximo de las dos anteriores
# sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan


import math

def linear(node):
    goal = (1,2,3,4,5,6,7,8,0)
    return sum([1 if node.state[i] != goal[i] else 0 for i in range(8)])

def manhattan(node):
    l = list(node.state)
    mhd = 0
    for m in range(0,3):
        for n in range(0,3):
            num = l[m*3+n]
            i = num/3; j = num%3 - 1
            if num == 0:
                i = 2; j = 2;
            mhd += abs(i-m) + abs(j-n);
 
    return mhd
                
def sqrt_manhattan(node):
    state = node.state
    mhd = manhattan(node)
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)

def linear15(node):
    goal = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)
    return sum([1 if node.state[i] != goal[i] else 0 for i in range(15)])

def manhattan15(node):
    l = list(node.state)
    mhd = 0
    for m in range(0,4):
        for n in range(0,4):
            num = l[m*4+n]
            i = num/4; j = num%4 - 1
            if num == 0:
                i = 3; j = 3;
            mhd += abs(i-m) + abs(j-n)
 
    return mhd
        

#### Comparar los tiempos de ejecución usando %%time y %%timeit y comentar los resultados

In [ ]:
####  Vamos a probar el mismo 8-puzzle utilizando el siguiente **estado inicial** que vimos en un ejemplo anterior. 
#              +---+---+---+
#              | 2 | 4 | 3 |
#              +---+---+---+
#              | 1 | 5 | 6 |
#              +---+---+---+
#              | 7 | 8 | H |
#              +---+---+---+

puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [ ]:
astar_search(puzle,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [ ]:
astar_search(puzle,linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [ ]:
astar_search(puzle,max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

¿Has notado diferencias en los tiempos de ejecución? ¿y en los resultados?
Vamos a medirlo. 
Se pueden hacer las pruebas directamente usando la clase Problem original y midiendo el tiempo con %time o %timeit según corresponda.

Te recomiendo usar las clases extendidas Problema_con_Analizados y Resuelve_puzle con la que puedes obtener mas información de la ejecución aparte del tiempo (nodos expandidos, ..) 

In [ ]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [ ]:
%%timeit
astar_search(puzzle_1, linear)

111 μs ± 1.75 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
%%timeit
astar_search(puzzle_1, manhattan)

419 μs ± 1.39 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

<Node (1, 2, 3, 4, 5, 6, 7, 8, 0)>

In [ ]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

129 ms ± 1.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

12 ms ± 108 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
uniform_cost_search(puzzle_1)
uniform_cost_search(puzzle_2)
uniform_cost_search(puzzle_3)

190 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
best_first_graph_search(puzzle_1, manhattan)
#best_first_graph_search(puzzle_2, manhattan)
#best_first_graph_search(puzzle_3, manhattan)

34 ms ± 379 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
best_first_graph_search(puzzle_1, sqrt_manhattan)
#best_first_graph_search(puzzle_2, sqrt_manhattan)
#best_first_graph_search(puzzle_3, sqrt_manhattan)

34.6 ms ± 701 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
best_first_graph_search(puzzle_1, max_heuristic)
#best_first_graph_search(puzzle_2, max_heuristic)
#best_first_graph_search(puzzle_3, max_heuristic)

35.2 ms ± 484 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
puzzle15_1 = Quince_Puzzle((12, 2, 6, 4, 1, 3, 7, 0, 5, 9, 14, 15, 13, 10, 8, 11))
puzzle15_2 = Quince_Puzzle((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 13, 14, 11, 0))
puzzle15_3 = Quince_Puzzle((1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 10, 15, 13, 14, 11, 0))

In [ ]:
%%timeit
astar_search(puzzle15_2, manhattan15)

174 μs ± 717 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
%%timeit
astar_search(puzzle15_2, linear15)

39.4 μs ± 322 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:

class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)

In [ ]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """
    p8=Ocho_Puzzle(estado_inicial)
    p8p=Problema_con_Analizados(p8)
    if p8.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


#### Observa los tiempos y comprueba que obtienes los resultados esperados  (según las clases teóricas) 

Si no es así (puede pasar) piensa cuál es la razón del comportamiento observado. 

- ¿Siempre A* es más rápida que otras búsquedas?  ¿Dentro de las pruebas con A*, cuál es la heurística que más ha tardado?
- ¿Tiene algún beneficio usar la raíz cuadrada?
- Si sabes que una h'1 es más informada que otra h'2, ¿siempre es más eficiente la búsqueda A* con h'1 que con h'2?  ¿lo has comprobado en los experimentos? 


In [ ]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))

In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 2
CPU times: total: 0 ns
Wall time: 0 ns


In [ ]:
%%time
resuelve_ocho_puzzle(estado_inicial,astar_search,linear)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 1. Nodos analizados: 2
CPU times: total: 0 ns
Wall time: 1.04 ms


Observa que el puzzle de 16 es notablemente más complejo que el puzzle de 8 debido a su mayor tamaño, y esto se refleja en los tiempos de ejecución a la hora de buscar soluciones. 

In [ ]:
def resuelve_quince_puzzle(estado_inicial, algoritmo, h=None):
    
    p15=Quince_Puzzle(estado_inicial)
    p15p=Problema_con_Analizados(p15)
    if p15.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p15p,h).solution()
        else: 
            sol= algoritmo(p15p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p15p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [ ]:
%%time
resuelve_quince_puzzle((1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15),best_first_graph_search, linear)
##Hay una solución en un movimiento. 

Solución: ['Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 1. Nodos analizados: 10
CPU times: total: 0 ns
Wall time: 111 μs


In [ ]:
%%time
resuelve_quince_puzzle((1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15),best_first_graph_search, manhattan)
## 

KeyboardInterrupt: 

In [ ]:
%%time
resuelve_quince_puzzle((1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15),astar_search,linear)
## Hay una solución en un movimiento ¿es mejor la heurística manhattan que la lineal?

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 1. Nodos analizados: 4
CPU times: total: 0 ns
Wall time: 1.01 ms


Si quieres explorar más el problema hay otras heurísticas mejores, por ejemplo, Walking Distance de Kenichiro Takahashi, o el uso de una base de datos auxiliar con patrones.


In [ ]:
%%time
resuelve_quince_puzzle((1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15),astar_search,manhattan)
##Sólo tiene que hacer un movimiento

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 4
CPU times: total: 0 ns
Wall time: 1.04 ms


In [ ]:
%%time
resuelve_quince_puzzle((1,2,3,4,5,6,7,8,9,10,11,12,13,0,14,15),astar_search,linear)
##Sólo tiene que hacer dos movimientos

Solución: ['Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 2. Nodos analizados: 8
CPU times: total: 0 ns
Wall time: 1.05 ms


In [ ]:
%%time
resuelve_quince_puzzle((1,2,3,4,5,6,7,8,9,10,11,12,13,0,14,15),astar_search,manhattan)

Solución: ['Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 2. Nodos analizados: 15
CPU times: total: 0 ns
Wall time: 1.05 ms


In [ ]:
%%time
resuelve_quince_puzzle((1,10,15,4,13,6,3,8,2,9,12,7,14,5,0,11),astar_search,linear)
## solución de 35 movimientos.

: 

La disposición de fichas anterior tiene una peculiaridad: los números en las filas, columnas y ambas diagonales principales suman 30. Esta disposición se llama un _Cuadrado Mágico_. Existen 7040 cuadrados mágicos diferentes, pero solo 3520 de ellos pueden resolverse con el espacio en blanco en la esquina inferior derecha. El que se muestra arriba es el único que puede resolverse en 35 movimientos, y no existen cuadrados mágicos que puedan resolverse con menos movimientos.
Un cuadrado mágico resoluble que se puede resolver con el espacio en blanco en la esquina inferior derecha no puede tener el espacio en blanco en esa misma esquina. Si coloreas el cuadrado de 4x4 como un tablero de ajedrez, todas las posibles posiciones del espacio en blanco en los cuadrados mágicos resolubles tienen un color diferente al de la posición de la esquina inferior derecha. Las cuatro posiciones posibles en la diagonal corresponden a 416 cuadrados mágicos cada una, y cada una de las otras cuatro posiciones corresponde a 464 cuadrados mágicos. En total, estos son exactamente los 3520 cuadrados mágicos resolubles.